In [8]:
# =============================================================================
# # Import Libraries
# =============================================================================
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz

In [9]:
# =============================================================================
# # Data Import
# =============================================================================
titanic_train = pd.read_csv('train.csv')

In [10]:
# =============================================================================
# Data cleaning
# =============================================================================
# Deleting inuseful labels and Dividing into features and Labels
titanic_train = titanic_train.drop([ 'name', 'cabin', 'ticket', 'embarked','fare'],axis=1)
## Fill Gaps (age)
titanic_train.age.fillna(titanic_train.age.dropna().mean(),inplace=True)
titanic_train.sex = np.where(titanic_train.sex == 'male', 1, 0)

X = titanic_train.values[:, 1:]
y = titanic_train.values[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
# =============================================================================
# # Split Class
# =============================================================================

# Split - info to use in nodes, represents how the sample is splitted
class Split:
    # Represents predicat: sample[feat]>=val
    def __init__(self, feat, val):
        self.feat = feat
        self.val = val
    # Compare if current object satisfies splitting condition 
    def match(self, obj):
        val = obj[self.feat]
        if isinstance(val, int) or isinstance(val, float):
            return val >= self.val
        else:
            return val == self.val
        

In [12]:
# =============================================================================
# # Fucnctions for the decision Tree
# =============================================================================
# Counts the number of each type of example in a dataset. Returns a dictionary label - count
def class_counts(rows):
    counts = {} 
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# Partitions a dataset by splitting condition. Returns two sub-datasets, one satisfies condition, one not.
def partition(X, split):
    X_true, X_false = [], []
    for row in X:
        if split.match(row):
            X_true.append(row)
        else:
            X_false.append(row)
    return X_true, X_false

# Calculate the Gini Impurity for a list of rows.
def gini(X):
    counts = class_counts(X)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(X))
        impurity -= prob_of_lbl**2
    return impurity

# Information Gain
def inform_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# "Find the best splitting condition by iterating over every feature / value and calculating the information gain.
def find_best_split(X):

    best_gain = 0 
    best_split = None 
    current_uncertainty = gini(X)
    n_features = len(X[0]) - 1 
    for col in range(n_features):
        values = set([row[col] for row in X]) 
        for val in values: 
            split = Split(col, val)
            X_true, X_false = partition(X, split)
            if len(X_true) == 0 or len(X_false) == 0:
                continue
            gain = inform_gain(X_true, X_false, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_split = gain, split
    return best_gain, best_split

# Get label with maximal probability in this sample
def get_prediction(prediction_dict):
    if prediction_dict:
        return max(prediction_dict.items(), key=operator.itemgetter(1))[0]
    else: return None

In [13]:
# =============================================================================
# # Leaf Class
# =============================================================================
# A Leaf node classifies data.
class Leaf:
    def __init__(self, X):
        self.predictions = class_counts(X)
        
# =============================================================================
# # Decision Node
# =============================================================================  
# Node of decision tree. Consists of splitting condition, two childs - true node and false node and current predictions
class Decision_Node:
   
    def __init__(self,
                 split,
                 true_branch,
                 false_branch, X):
        self.split = split
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.predictions = class_counts(X)

In [14]:
# =============================================================================
# # Decision Tree
# =============================================================================
class decision_tree(BaseEstimator, ClassifierMixin):

    def __init__(self, max_depth = 5):
        self.max_depth = max_depth
        self.tree = None
        self.pruned_tree = None
        
#Builds the tree.
    def make_tree(self, X):
        gain, split = find_best_split(X)
        if gain == 0:
            return Leaf(X)
        X_true, X_false = partition(X, split)
        true_branch = self.make_tree(X_true)
        false_branch = self.make_tree(X_false)
        return Decision_Node(split, true_branch, false_branch, X)

    def fit(self, X):
        self.tree = self.make_tree(X)
        return self

    def print_tree(self, node, spacing="", level=0):
        if isinstance(node, Leaf):
            print(spacing + "Predict", node.predictions)
            return
        if level<self.max_depth:
            print(spacing + str(node.split.visualize()))
            print(spacing + '--> True:')
            self.print_tree(node.true_branch, spacing + "  ", level+1)
            print(spacing + '--> False:')
            self.print_tree(node.false_branch, spacing + "  ", level+1)
        else: 
            print(spacing + "Predict", node.predictions)
            return

# Go through tree
    def classify_one(self, row, node, level=0):
        if isinstance(node, Leaf):
            return node.predictions
        while level<self.max_depth:
            if node.split.match(row):
                return self.classify_one(row, node.true_branch, level+1)
            else:
                return self.classify_one(row, node.false_branch, level+1)
        return node.predictions

    def predict(self, X):
        pr = []
        for row in X:
            pr.append(get_prediction(self.classify_one(row, self.tree)))
        return pr


In [31]:
# =============================================================================
# # Fit & Predict
# =============================================================================

descisionTree = decision_tree()
# Fit function call
descisionTree.fit(X_train, y_train)

decision_tree(threshold=None)

In [32]:
# =============================================================================
# # result with my classifier
# =============================================================================
predictions = descisionTree.predict(X_test)
conf_matr = confusion_matrix(y_test, predictions)
# Print accuracy and conf. matrix:
accuracy = accuracy_score(y_test, predictions)
print('Accuracy: ','{0:.4f}%'.format(accuracy * 100))
print('Confussion Matrix:\n', conf_matr)

Accuracy:  82.6816%
Confussion Matrix:
 [[105  16]
 [ 15  43]]


In [8]:
# =============================================================================
# # Compare with SKLEARN DecisionTreeClassifier:
# =============================================================================
skDecisionTree = DecisionTreeClassifier()
# Fit function call
skDecisionTree.fit(X, y)
predictions_sk = skDecisionTree.predict(X_test)
conf_matr_sk = confusion_matrix(y_test, predictions_sk)
# Print accuracy and conf. matrix:
accuracy = accuracy_score(y_test, predictions_sk)
print('Accuracy: ','{0:.4f}%'.format(accuracy * 100))
print('Confussion Matrix:\n', conf_matr_sk)


Accuracy:  98.3240%
Confussion Matrix:
 [[102   0]
 [  3  74]]
